# Ejercicio Formativo 2 Capítulo 6


## Importando Librerías


In [1]:
import json
import sqlite3

## Misión 1: Modelación de entidades


In [2]:
with open("football_results.json", encoding="utf8") as football_results_file:
    football_results = json.load(football_results_file)

In [3]:
for index in range(5):
    print(football_results[index])

{'league': 'Bundesliga', 'season': 2014, 'date': '2014-08-22 19:30:00', 'team': 'Bayern Munich', 'h_a': 'h', 'result': 'w', 'pts': 3, 'goals_scored': 2, 'goals_missed': 1, 'deep_passes': 5, 'deep_passes_allowed': 4, 'ppda': 9.625, 'oppda': 21.85}
{'league': 'Bundesliga', 'season': 2014, 'date': '2014-08-30 17:30:00', 'team': 'Bayern Munich', 'h_a': 'a', 'result': 'd', 'pts': 1, 'goals_scored': 1, 'goals_missed': 1, 'deep_passes': 10, 'deep_passes_allowed': 1, 'ppda': 4.7560975609756095, 'oppda': 17.695652173913043}
{'league': 'Bundesliga', 'season': 2014, 'date': '2014-09-13 14:30:00', 'team': 'Bayern Munich', 'h_a': 'h', 'result': 'w', 'pts': 3, 'goals_scored': 2, 'goals_missed': 0, 'deep_passes': 13, 'deep_passes_allowed': 3, 'ppda': 5.0606060606060606, 'oppda': 16.96153846153846}
{'league': 'Bundesliga', 'season': 2014, 'date': '2014-09-20 14:30:00', 'team': 'Bayern Munich', 'h_a': 'a', 'result': 'd', 'pts': 1, 'goals_scored': 0, 'goals_missed': 0, 'deep_passes': 6, 'deep_passes_all

Se modelarán tres entidades:

- **Leagues**: Representa una liga. Tiene los siguientes campos:
    - `lid`: Identificador único de la liga. Tendra un campo del tipo INTEGER.
    - `name`: Nombre de la liga. Tendra un campo del tipo TEXT.
- **Teams**: Representa un equipo. Tiene los siguientes campos:
    - `tid`: Identificador único del equipo. Tendra un campo del tipo INTEGER.
    - `name`: Nombre del equipo. Tendra un campo del tipo TEXT.
- **Games**: Representa un partido. Tiene los siguientes campos:
    - `gid`: Identificador único del partido. Tendra un campo del tipo INTEGER.
    - `league_id`: Identificador de la liga a la que pertenece el partido. Tendra un campo del tipo INTEGER.
    - `season`: Temporada en la que se jugó el partido. Tendra un campo del tipo INTEGER.
    - `date`: Fecha en la que se jugó el partido. Tendra un campo del tipo DATE.
    - `team_id`: Identificador del equipo que jugó el partido. Tendra un campo del tipo INTEGER.
    - `h_a`: Indica si el equipo es local (h) o visitante (a). Tendra un campo del tipo TEXT.
    - `result`: Resultado del partido. Tendra un campo del tipo TEXT.
    - `pts`: Puntos obtenidos por el equipo. Tendra un campo del tipo INTEGER.
    - `goals_scored`: Goles anotados por el equipo analizado. Tendra un campo del tipo INTEGER.
    - `goals_missed`: Goles recibidos por el equipo analizado. Tendra un campo del tipo INTEGER.
    - `deep_passes`: Pases exitosos realizados por el equipo analizado en los  últimos 20 metros de la cancha (pases profundos). Tendra un campo del tipo INTEGER.
    - `deep_passes_allowed`: pases profundos exitosos realizados por el contrincante. Tendra un campo del tipo INTEGER.
    - `ppda`: Indice de presión defensiva. Tendra un campo del tipo FLOAT.
    - `oppda`: Indice de presión defensiva del equipo contrincante. Tendra un campo del tipo FLOAT.

Ahora se procederá a crear las tablas

In [4]:
connection = sqlite3.connect("football_results.db")
cursor = connection.cursor()

In [5]:
cursor.execute("CREATE TABLE IF NOT EXISTS Leagues(lid INTEGER PRIMARY KEY, name TEXT)")
cursor.execute("CREATE TABLE IF NOT EXISTS Teams(tid INTEGER PRIMARY KEY, name TEXT)")
cursor.execute(
    "CREATE TABLE IF NOT EXISTS Games(gid INTEGER PRIMARY KEY, league_id INTEGER, season INTEGER, date DATE, team_id INTEGER, h_a TEXT, result TEXT, pts INTEGER, goals_scored INTEGER, goals_missed INTEGER, deep_passes INTEGER, deep_passes_allowed INTEGER, ppda FLOAT, oppda FLOAT, FOREIGN KEY(league_id) REFERENCES Leagues, FOREIGN KEY(team_id) REFERENCES Teams)"
)

Al igual que para el ejercicio anterior, se creará una base de datos en SQLite y se crearán las tablas correspondientes. La diferencia en este ejercicio es que hay algunos campos que hacen referencia a otras tablas. Por ejemplo, el campo `league_id` de la tabla `Games` hace referencia al campo `lid` de la tabla `Leagues`. Para esto, se debe crear una llave foránea que haga referencia a la tabla `Leagues`. Lo mismo se debe hacer para el campo `team_id` de la tabla `Games`, que hace referencia al campo `tid` de la tabla `Teams`. Por ello se añade el campo `FOREIGN KEY` en la creación de la tabla `Games`.

In [6]:
connection.commit()
connection.close()

## Misión 2: Carga de datos


In [7]:
connection = sqlite3.connect("football_results.db")
cursor = connection.cursor()

lid = 1
tid = 1
leagues = {}
teams = {}

for game in football_results:
    league_name = game["league"]
    if league_name not in leagues:
        cursor.execute("INSERT INTO Leagues(name) VALUES(?)", (league_name,))
        leagues[league_name] = lid
        lid += 1

    team_name = game["team"]
    if team_name not in teams:
        cursor.execute("INSERT INTO Teams(name) VALUES(?)", (team_name,))
        teams[team_name] = tid
        tid += 1

for game in football_results:
    league_name = game["league"]
    team_name = game["team"]
    season = game["season"]
    date = game["date"]
    h_a = game["h_a"]
    result = game["result"]
    pts = game["pts"]
    goals_scored = game["goals_scored"]
    goals_missed = game["goals_missed"]
    deep_passes = game["deep_passes"]
    deep_passes_allowed = game["deep_passes_allowed"]
    ppda = game["ppda"]
    oppda = game["oppda"]
    data = (
        leagues[league_name],
        season,
        date,
        teams[team_name],
        h_a,
        result,
        pts,
        goals_scored,
        goals_missed,
        deep_passes,
        deep_passes_allowed,
        ppda,
        oppda,
    )

    cursor.execute(
        "INSERT INTO Games(league_id, season, date, team_id, h_a, result, pts, goals_scored, goals_missed, deep_passes, deep_passes_allowed, ppda, oppda) VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?)",
        data,
    )

connection.commit()
connection.close()

En primer lugar se crean los diccionarios para las entidades `Leagues` y `Teams`, además se inicializan los identificadores de las entidades en 1.

Desglozamos el código para hacerlo más entendible:

```python
for game in football_results:
    league_name = game["league"]
    if league_name not in leagues:
        cursor.execute("INSERT INTO Leagues(name) VALUES(?)", (league_name,))
        leagues[league_name] = lid
        lid += 1

    team_name = game["team"]
    if team_name not in teams:
        cursor.execute("INSERT INTO Teams(name) VALUES(?)", (team_name,))
        teams[team_name] = tid
        tid += 1
```

Lo que hace este segmento de código es recorrer la lista `football_results` y por cada partido, se verifica si la liga del partido ya se encuentra en el diccionario `leagues`. Si no se encuentra, se inserta la liga en la tabla `Leagues` y se añade al diccionario `leagues`. Lo mismo se hace para los equipos, verificando si el equipo ya se encuentra en el diccionario `teams`. Si no se encuentra, se inserta el equipo en la tabla `Teams` y se añade al diccionario `teams`. Recordar que por tabla se refiere a la base de datos.


```python
for game in football_results:
    league_name = game["league"]
    team_name = game["team"]
    season = game["season"]
    date = game["date"]
    h_a = game["h_a"]
    result = game["result"]
    pts = game["pts"]
    goals_scored = game["goals_scored"]
    goals_missed = game["goals_missed"]
    deep_passes = game["deep_passes"]
    deep_passes_allowed = game["deep_passes_allowed"]
    ppda = game["ppda"]
    oppda = game["oppda"]
    data = (
        leagues[league_name],
        season,
        date,
        teams[team_name],
        h_a,
        result,
        pts,
        goals_scored,
        goals_missed,
        deep_passes,
        deep_passes_allowed,
        ppda,
        oppda,
    )

    cursor.execute(
        "INSERT INTO Games(league_id, season, date, team_id, h_a, result, pts, goals_scored, goals_missed, deep_passes, deep_passes_allowed, ppda, oppda) VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?)",
        data,
    )
```

Para esta parte del código nuevamente se iterará sobre la lista `football_results` y por cada partido se obtienen los datos necesarios para insertar en la tabla `Games`. Se obtienen los identificadores de la liga y del equipo del partido, que se encuentran en los diccionarios `leagues` y `teams`, respectivamente. Luego se insertan los datos en la tabla `Games`.

Como se pudo observar en este ejercicio y en el ejercicio anterior, para realizar las operaciones de creación de tablas e inserción de datos se utiliza generalmente el mismo patrón de código, lo que cambia es principalmente la estructura de las tablas y los datos a insertar, lo cual esta determinado por el problema a resolver y la interpretación de los datos. También se recalca que la manera de insertar los datos queda al criterio de cada uno, ya que por ejemplo en mi caso se realizaron varias iteraciones para insertar los datos, pero esto no es una regla, se puede insertar los datos de una sola vez si se desea.